In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

In [2]:
# Import train and test data
df_train = pd.read_csv('C:\Kaggle\Upvotes/train_upvotes.csv')
df_test = pd.read_csv('C:\Kaggle\Upvotes/test_upvotes.csv')

In [3]:
df_train.head(10)

,ID,Tag,Reputation,Answers,Username,Views,Upvotes
0,52664,a,3942.0,2.0,155623,7855.0,42.0
1,327662,a,26046.0,12.0,21781,55801.0,1175.0
2,468453,c,1358.0,4.0,56177,8067.0,60.0
3,96996,a,264.0,3.0,168793,27064.0,9.0
4,131465,c,4271.0,4.0,112223,13986.0,83.0
5,345568,r,440.0,1.0,23866,5577.0,4.0
6,376865,c,147.0,1.0,10759,17184.0,17.0
7,467378,j,2269.0,2.0,54623,312.0,3.0
8,393640,j,111.0,2.0,172926,53738.0,79.0
9,130054,c,2749.0,1.0,10189,227.0,0.0


In [4]:
df_train.shape

(330045, 7)

In [5]:
df_test.shape

(141448, 6)

In [6]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 330045 entries, 0 to 330044
Data columns (total 7 columns):
ID            330045 non-null int64
Tag           330045 non-null object
Reputation    330045 non-null float64
Answers       330045 non-null float64
Username      330045 non-null int64
Views         330045 non-null float64
Upvotes       330045 non-null float64
dtypes: float64(4), int64(2), object(1)
memory usage: 17.6+ MB


In [7]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141448 entries, 0 to 141447
Data columns (total 6 columns):
ID            141448 non-null int64
Tag           141448 non-null object
Reputation    141448 non-null float64
Answers       141448 non-null float64
Username      141448 non-null int64
Views         141448 non-null float64
dtypes: float64(3), int64(2), object(1)
memory usage: 6.5+ MB


In [8]:
df_train.describe()

,ID,Reputation,Answers,Username,Views,Upvotes
count,330045.000000,3.300450e+05,330045.000000,330045.000000,3.300450e+05,330045.000000
mean,235748.682789,7.773147e+03,3.917672,81442.888803,2.964507e+04,337.505358
std,136039.418471,2.706141e+04,3.579515,49215.100730,8.095646e+04,3592.441135
min,1.000000,0.000000e+00,0.000000,0.000000,9.000000e+00,0.000000
25%,117909.000000,2.820000e+02,2.000000,39808.000000,2.594000e+03,8.000000
50%,235699.000000,1.236000e+03,3.000000,79010.000000,8.954000e+03,28.000000
75%,353620.000000,5.118000e+03,5.000000,122559.000000,2.687000e+04,107.000000
max,471493.000000,1.042428e+06,76.000000,175738.000000,5.231058e+06,615278.000000


In [9]:
df_train = df_train.drop(df_train[df_train.Views > 3000000].index)

In [10]:
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn import linear_model

In [11]:
le = LabelEncoder()
df_train['Tag'] = le.fit_transform(df_train['Tag'])
df_train = df_train.drop(['ID'], axis=1)
df_train = df_train.drop(['Username'], axis=1)
target = df_train['Upvotes']

In [12]:
test_id = df_test['ID']
df_test['Tag'] = le.transform(df_test['Tag'])
df_test = df_test.drop(['ID'], axis=1)
df_test = df_test.drop(['Username'], axis=1) 

In [13]:
df_train.shape

(330032, 5)

In [14]:
df_test.shape

(141448, 4)

In [15]:
df_train = df_train.drop(['Upvotes'], axis=1)

In [16]:
from sklearn.preprocessing import Binarizer
bn = Binarizer(threshold=7)
pd_watched = bn.transform([df_train['Answers']])[0]
df_train['pd_watched'] = pd_watched

pd_watched = bn.transform([df_test['Answers']])[0]
df_test['pd_watched'] = pd_watched

In [17]:
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(df_train, target, test_size = 0.22,random_state =205)
sc_X = StandardScaler()
x_train = sc_X.fit_transform(x_train)
x_val = sc_X.transform(x_val)

In [18]:
from sklearn.preprocessing import PolynomialFeatures

In [19]:
poly_reg = PolynomialFeatures(degree = 4,interaction_only=False, include_bias=True)
X_poly = poly_reg.fit_transform(x_train)
poly_reg.fit(x_train, y_train)
lin_reg_1 = linear_model.LassoLars(alpha=0.021,max_iter=150)
lin_reg_1.fit(X_poly, y_train)

LassoLars(alpha=0.021, copy_X=True, eps=2.220446049250313e-16,
     fit_intercept=True, fit_path=True, max_iter=150, normalize=True,
     positive=False, precompute='auto', verbose=False)

In [20]:
# predicitng 
pred_val = lin_reg_1.predict(poly_reg.fit_transform(x_val))

print(r2_score(y_val, pred_val))

0.9034378793327871


In [21]:
df_test = sc_X.transform(df_test)

pred_test = lin_reg_1.predict(poly_reg.transform(df_test))
pred_test = abs(pred_test)

In [22]:
submission = pd.DataFrame({'ID': test_id,
                           'Upvotes':pred_test
                           })

submission.to_csv("upvotes.csv",index=False)